### Segmenting and Clustering Neighborhoods in Toronto
IBM Data Science Professional Certificate - Applied Data Capstone 
##### By: Joe Lavond
---

##### Set Up 
---

##### If needed - change cell from 'markdown' to 'code'
!pip install bs4

In [205]:
import pandas as pd
import numpy as np

##### Working  
---

In [206]:
import requests
from bs4 import BeautifulSoup

url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(url, 'html.parser')
soup = soup.html.body.find('div',{'class':'mw-parser-output'})
contents = soup.table.find_all('p')

content_text = []
for elem in contents:
    content_text.append(elem.text)
#print(content_text)

In [207]:
all_info = []
for elem in content_text:
    info = [elem[0:3]]
    
    rest = elem[3:].replace('\n', '')
    rest = rest.replace(' / ', ', ').split(')')[0].split('(')
    while len(rest) < 2:
        rest.append('')
        
    all_info.append(info + rest)   

In [208]:
data = pd.DataFrame(all_info)
data.columns = ['postal_code', 'borough', 'neighborhood']
data = data[data['borough'] != 'Not assigned']
data.head()

,postal_code,borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,"Queen's Park, Ontario Provincial Government",


In [209]:
data.shape

(103, 3)

In [210]:
# Merge Location Data
lat_long = pd.read_csv('Geospatial_Coordinates.csv')
lat_long.columns = ['postal_code', 'lat', 'long']

data = data.merge(lat_long, on = 'postal_code')
data

,postal_code,borough,neighborhood,lat,long
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,"Queen's Park, Ontario Provincial Government",,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
